# Sesión 5 – Orquestador Multi-Agente

Demuestra una sencilla canalización de dos agentes (Investigador -> Editor) utilizando Foundry Local.


### Explicación: Instalación de Dependencias
Instala `foundry-local-sdk` y `openai`, necesarios para el acceso a modelos locales y completaciones de chat. Idempotente.


# Escenario
Implementa un patrón de orquestación mínimo con dos agentes:
- **Agente Investigador** recopila puntos clave y concisos basados en hechos.
- **Agente Editor** reescribe para mayor claridad ejecutiva.

Demuestra memoria compartida por agente, paso secuencial de resultados intermedios y una función de canalización simple. Es ampliable a más roles (por ejemplo, Crítico, Verificador) o ramas paralelas.

**Variables de Entorno:**
- `FOUNDRY_LOCAL_ALIAS` - Modelo predeterminado a usar (por defecto: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Modelo principal del agente (anula ALIAS)
- `AGENT_MODEL_EDITOR` - Modelo del agente editor (por defecto, el principal)

**Referencia del SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Cómo funciona:**
1. **FoundryLocalManager** inicia automáticamente el servicio Foundry Local.
2. Descarga y carga el modelo especificado (o utiliza la versión en caché).
3. Proporciona un punto de interacción compatible con OpenAI.
4. Cada agente puede usar un modelo diferente para tareas especializadas.
5. La lógica de reintento integrada maneja fallos transitorios de manera eficiente.

**Características Clave:**
- ✅ Descubrimiento e inicialización automática del servicio.
- ✅ Gestión del ciclo de vida del modelo (descarga, caché, carga).
- ✅ Compatibilidad con el SDK de OpenAI para una API familiar.
- ✅ Soporte para múltiples modelos para especialización de agentes.
- ✅ Manejo robusto de errores con lógica de reintento.
- ✅ Inferencia local (no se requiere API en la nube).


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Explicación: Importaciones principales y tipado
Introduce dataclasses para el almacenamiento de mensajes de agentes y sugerencias de tipado para mayor claridad. Importa el gestor local de Foundry + cliente de OpenAI para las acciones posteriores del agente.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Explicación: Inicialización del Modelo (Patrón SDK)
Utiliza el SDK de Python de Foundry Local para una gestión robusta de modelos:
- **FoundryLocalManager(alias)** - Inicia automáticamente el servicio y carga el modelo mediante un alias.
- **get_model_info(alias)** - Resuelve el alias al ID concreto del modelo.
- **manager.endpoint** - Proporciona el endpoint del servicio para el cliente de OpenAI.
- **manager.api_key** - Proporciona la clave API (opcional para uso local).
- Admite modelos separados para diferentes agentes (principal vs editor).
- Lógica de reintento incorporada con retroceso exponencial para mayor resiliencia.
- Verificación de conexión para garantizar que el servicio esté listo.

**Patrón clave del SDK:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Gestión del Ciclo de Vida:**
- Los managers se almacenan globalmente para una limpieza adecuada.
- Cada agente puede usar un modelo diferente para especialización.
- Descubrimiento automático de servicios y manejo de conexiones.
- Reintento elegante con retroceso exponencial en caso de fallos.

Esto asegura una inicialización adecuada antes de que comience la orquestación de agentes.

**Referencia:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Explicación: Clases de Agente y Memoria
Define un `AgentMsg` ligero para entradas de memoria y un `Agent` que encapsula:
- **Rol del sistema** - La personalidad e instrucciones del agente
- **Historial de mensajes** - Mantiene el contexto de la conversación
- **Método act()** - Ejecuta acciones con un manejo adecuado de errores

El agente puede usar diferentes modelos (principal vs editor) y mantiene un contexto aislado por agente. Este patrón permite:
- Persistencia de memoria entre acciones
- Asignación flexible de modelos por agente
- Aislamiento y recuperación de errores
- Encadenamiento y orquestación sencillos


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Explicación: Pipeline Orquestado
Crea dos agentes especializados:
- **Investigador**: Utiliza el modelo principal, recopila información factual
- **Editor**: Puede usar un modelo diferente (si está configurado), refina y reescribe

La función `pipeline`:
1. El Investigador recopila información en bruto
2. El Editor la refina para obtener un resultado listo para ejecutivos
3. Devuelve tanto los resultados intermedios como los finales

Este patrón permite:
- Especialización de modelos (diferentes modelos para diferentes roles)
- Mejora de la calidad mediante procesamiento en múltiples etapas
- Rastreabilidad de la transformación de la información
- Fácil ampliación a más agentes o procesamiento en paralelo


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Explicación: Ejecución del Pipeline y Resultados
Ejecuta el pipeline de múltiples agentes en una pregunta centrada en cumplimiento y latencia para demostrar:
- Transformación de información en múltiples etapas
- Especialización y colaboración entre agentes
- Mejora de la calidad del resultado mediante refinamiento
- Rastreabilidad (se conservan tanto los resultados intermedios como los finales)

**Estructura del Resultado:**
- `question` - Consulta original del usuario
- `research` - Resultado de investigación en bruto (puntos factuales)
- `final` - Resumen ejecutivo refinado
- `models` - Modelos utilizados en cada etapa

**Ideas de Extensión:**
1. Añadir un agente Crítico para revisión de calidad
2. Implementar agentes de investigación paralelos para diferentes aspectos
3. Añadir un agente Verificador para comprobación de hechos
4. Usar diferentes modelos para distintos niveles de complejidad
5. Implementar bucles de retroalimentación para mejora iterativa


### Avanzado: Configuración personalizada de agentes

Intenta personalizar el comportamiento del agente modificando las variables de entorno antes de ejecutar la celda de inicialización:

**Modelos disponibles:**
- Usa `foundry model ls` en el terminal para ver todos los modelos disponibles
- Ejemplos: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, etc.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
